## OSDA. Project. Data Analysis

### Вспомогательные функции

Для начала импортируем нужные библиотеки и создадим список всех обучающих и тестовых выборок.

Я буду использовать датасеты по игре "Крестики-нолики", которые были представлены в задании.

In [ ]:
import numpy as np
import pandas as pd
import time
from scipy import stats

In [ ]:
list_train = []
list_test = []
for i in range(1, 11):
    list_train.append(pd.read_csv("train" + str(i) + ".csv"))
    list_test.append(pd.read_csv("test" + str(i) + ".csv"))

Реализуем вспомогательные функции.

get_intersection(a, b) находит пересечения двух векторов. Возвращает список значений пересекающихся элементов и список их порядковых номеров.

step_lazyFCA(X_train, y_train, num_train_array, test_array) выполняет шаг алгоритма LazyFCA.

Возвращает 0, если нет пересечения, удовлетворяющего условию алгоритма LazyFCA.
Иначе возвращает 1, если train[j] имеет положительный тип, возвращает -1, если отрицательный тип.

get_metrics(y_test, y_pred) вычисляет значения всех необходимых метрик.

- Accuracy
- True Positive
- True Negative
- False Positive
- False Negative
- True Positive Rate
- True Negative Rate
- Negative Predictive Value
- False Positive Rate
- False Discovery Rate
- Precision
- Recall

Для каждой функции приведен пример работы.

In [ ]:
def get_intersection(a, b):
    N = len(a)
    intersection = []
    num_columns = []
    for i in range(N):
        if a[i] == b[i]:
            intersection.append(a[i])
            num_columns.append(i)
    return intersection, num_columns

In [ ]:
df = pd.read_csv("train1.csv")
dtest = pd.read_csv("test1.csv")
X_train = np.array(df.iloc[:, 0:9])
y_train = np.array(df.iloc[:, 9])
X_test = np.array(dtest.iloc[:, 0:9])
y_test = np.array(dtest.iloc[:, 9])
a = X_train[0]
b = X_test[0]
print(a, b)
print(get_intersection(a, b))

['x' 'x' 'x' 'x' 'o' 'o' 'x' 'o' 'o'] ['x' 'x' 'x' 'x' 'o' 'o' 'o' 'x' 'o']
(['x', 'x', 'x', 'x', 'o', 'o', 'o'], [0, 1, 2, 3, 4, 5, 8])


In [ ]:
# Вернуть 0, если нет пересечения, удовлетворяющего условию алгоритма LazyFCA.
# Иначе вернуть 1, если положительный тип, вернуть -1, если отрицательный тип.

def step_lazyFCA(X_train, y_train, num_train_array, test_array):
    intersection, columns = get_intersection(X_train[num_train_array], test_array)
    N = len(X_train)
    M = len(intersection)
    if y_train[num_train_array] == 'positive':
        checked = 'negative'
    else:
        checked = 'positive'
    for i in range(N):
        if y_train[i] == checked:
            ind = True
            for j in range(M):
                if X_train[i][columns[j]] != intersection[j]:
                    ind = False
            if ind:
                return 0
    if checked == 'negative':
        return 1
    else:
        return -1

In [ ]:
test_array = X_test[0]
step_lazyFCA(X_train, y_train, 0, test_array)

1

In [ ]:
def get_metrics(y_test, y_pred):
    N = len(y_test)
    num_TP = 0
    num_TN = 0
    num_FP = 0
    num_FN = 0
    for i in range(N):
        if y_test[i] == 'positive':
            if y_pred[i] == 'positive':
                num_TP += 1
            else:
                num_FN += 1
        else:
            if y_pred[i] == 'positive':
                num_FP += 1
            else:
                num_TN += 1
    TP = num_TP / N
    TN = num_TN / N
    FP = num_FP / N
    FN = num_FN / N
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    NPV = TN / (TN + FN)
    FDR = FP / (TP + FP)
    accuracy = TP + TN
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return accuracy, TP, TN, FP, FN, TPR, TNR, FPR, NPV, FDR, precision, recall

In [ ]:
a = np.array(['positive', 'positive', 'positive', 'negative', 'negative'])
b = np.array(['positive', 'positive', 'negative', 'negative', 'positive'])
np.round(get_metrics(a, b), 3)

array([0.6  , 0.4  , 0.2  , 0.2  , 0.2  , 0.667, 0.5  , 0.5  , 0.5  ,
       0.333, 0.667, 0.667])

### Реализация алгоритмов обучения

Все алгоритмы принимают на вход массив метрик и классов для обучающей выборки и массив метрик для тестовой выборки. 

Возвращают предсказание классов тестовой выборки.

1. Базовый алгоритм LazyFCA. Метод реализован на основе генераторов. 

Для каждого из объекта из плюс-контекста находим пересечение его описания и описания объекта из тестовой выборки. Если оно не встречается ни в одном из описаний минус-примеров, то увеличиваем на 1 число "аргументов в пользу того, что объект из тестовой выборки положительный".

Для каждого из объекта из минус-контекста находим пересечение его описания и описания объекта из тестовой выборки. Если оно не встречается ни в одном из описаний плюс-примеров, то увеличиваем на 1 число "аргументов в пользу того, что объект из тестовой выборки отрицательный".

Далее сравниваем количество "аргументов". Если больше "аргументов" в пользу того, что объект тестовой выборки положительный, принимаем y_pred[i] = '+'. Если больше "аргументов" в пользу того, что объект тестовой выборки отрицательный, принимаем y_pred[i] = '-'. Если же количество аргументов равно, то принимаем "+" с вероятностью p, которую можно передать как аргумент.

2. Средняя мощность пересечения.

Вычисляет среднюю мощность пересечения всех объектов плюс-контекста с объектом тестовой выборки и среднюю мощность пересечения всех объектов минус-контекста с объектом тестовой выборки. Принимаем ту гипотезу, для которой средняя мощность пересечения больше.

3. Мощность пересечения k ближайших объектов.

Определим близость векторов как мощность их пересечения (чем больше мощность пересечения, тем объекты ближе). Для каждого из плюс и минус контекстов найдем k объектов, наиболее близких в объекту из тестовой выборки, и сравним средние мощности для k ближайших объектов. Принимаем ту гипотезу, для которой средняя мощность пересечения больше. 

4. Продвинутый алгоритм LazyFCA.

Это ансамбль алгоритмов №1 и №3. Сначала применяем алгоритм №1. Если разница по модулю "аргументов" в пользу того, что объект тестовой выборки принимает положительное или отрицательное значение, достигает значения d, то мы принимаем этот результат. В противном случае, если число аргументов в пользу каждого из классов примерно равно, применяем решение с помощью алгоритма №3.

In [ ]:
def baseline_lazyFCA(X_train, y_train, X_test, p=0.5):
    N = len(X_test)
    y_pred = []
    for i in range(N):
        num_pos = 0
        num_neg = 0
        for j in range(len(y_train)):
            step = step_lazyFCA(X_train, y_train, j, X_test[i])
            if step == 1:
                num_pos += 1
            elif step == -1:
                num_neg += 1
        if num_pos > num_neg:
            y_pred.append('positive')
        elif num_pos < num_neg:
            y_pred.append('negative')
        else:
            rand = stats.bernoulli(p).rvs()
            if rand == 1:
                y_pred.append('positive')
            else:
                y_pred.append('negative')
    return np.array(y_pred)

def avg_cardinality(X_train, y_train, X_test):
    N = len(X_test)
    y_pred = []
    for i in range(N):
        N_pos = 0
        N_neg = 0
        sum_pos = 0
        sum_neg = 0
        for j in range(len(y_train)):
            if y_train[j] == 'positive':
                N_pos += 1
                sum_pos += len(get_intersection(X_test[i], X_train[j])[0])
            else:
                N_neg += 1
                sum_neg += len(get_intersection(X_test[i], X_train[j])[0])
        avg_pos = sum_pos / N_pos
        avg_neg = sum_neg / N_neg
        if avg_pos > avg_neg:
            y_pred.append('positive')
        else:
            y_pred.append('negative')
    return np.array(y_pred)

def top_k_cardinality(X_train, y_train, X_test, k=10):
    N = len(X_test)
    y_pred = []
    for i in range(N):
        pos_card = []
        neg_card = []
        for j in range(len(y_train)):
            if y_train[j] == 'positive':
                pos_card.append(len(get_intersection(X_test[i], X_train[j])[0]))
            else:
                neg_card.append(len(get_intersection(X_test[i], X_train[j])[0]))
        pos_card = sorted(pos_card)
        neg_card = sorted(neg_card)
        k_pos_card = np.array(pos_card[-k:])
        k_neg_card = np.array(neg_card[-k:])
        avg_pos = np.mean(k_pos_card)
        avg_neg = np.mean(k_neg_card)
        if avg_pos > avg_neg:
            y_pred.append('positive')
        else:
            y_pred.append('negative')
    return np.array(y_pred)

def smart_lazy_FCA(X_train, y_train, X_test, k=10, d=5):
    N = len(X_test)
    y_pred = []
    for i in range(N):
        num_pos = 0
        num_neg = 0
        for j in range(len(y_train)):
            step = step_lazyFCA(X_train, y_train, j, X_test[i])
            if step == 1:
                num_pos += 1
            elif step == -1:
                num_neg += 1
        if num_pos >= num_neg + d:
            y_pred.append('positive')
        elif num_neg >= num_pos + d:
            y_pred.append('negative')
        else:
            pos_card = []
            neg_card = []
            for j in range(len(y_train)):
                if y_train[j] == 'positive':
                    pos_card.append(len(get_intersection(X_test[i], X_train[j])[0]))
                else:
                    neg_card.append(len(get_intersection(X_test[i], X_train[j])[0]))
            pos_card = sorted(pos_card)
            neg_card = sorted(neg_card)
            k_pos_card = np.array(pos_card[-k:])
            k_neg_card = np.array(neg_card[-k:])
            avg_pos = np.mean(k_pos_card)
            avg_neg = np.mean(k_neg_card)
            if avg_pos > avg_neg:
                y_pred.append('positive')
            else:
                y_pred.append('negative')
    return np.array(y_pred)


### Результаты на датасетах про крестики-нолики.

Найдем среднее время работы и среднее значение метрик качества для всех реализованных алгоритмов.

Будем использовать 10 датасетов для обучения и теста, которые представлены в материалах к заданию (Tic-Tac-Toe).

In [ ]:
RES = np.zeros((4, 13)) 

# Вычисляем метрики и время работы для всех четырех алгоритмах на 10 датасетах.
for i in range(10):
    df = list_train[i]
    dtest = list_test[i]
    X_train = np.array(df.iloc[:, 0:9])
    y_train = np.array(df.iloc[:, 9])
    X_test = np.array(dtest.iloc[:, 0:9])
    y_test = np.array(dtest.iloc[:, 9])
    
    start = time.time()
    y_pred = baseline_lazyFCA(X_train, y_train, X_test)
    end = time.time()
    RES[0][:12] += get_metrics(y_test, y_pred)
    RES[0][12] += (end - start)
    
    start = time.time()
    y_pred = avg_cardinality(X_train, y_train, X_test)
    end = time.time()
    RES[1][:12] += get_metrics(y_test, y_pred)
    RES[1][12] += (end - start)
    
    start = time.time()
    y_pred = top_k_cardinality(X_train, y_train, X_test)
    end = time.time()
    RES[2][:12] += get_metrics(y_test, y_pred)
    RES[2][12] += (end - start)
    
    start = time.time()
    y_pred = smart_lazy_FCA(X_train, y_train, X_test)
    end = time.time()
    RES[3][:12] += get_metrics(y_test, y_pred)
    RES[3][12] += (end - start)

# Делим на число экспериментов, чтобы получить среднее.    
RES = RES / 10
df_res = pd.DataFrame(RES.T)
df_res

,0,1,2,3
0,0.940541,0.658806,0.989572,0.953244
1,0.653474,0.427801,0.652375,0.653474
2,0.287067,0.231004,0.337197,0.299770
3,0.059459,0.115522,0.009329,0.046756
4,0.000000,0.225672,0.001099,0.000000
5,1.000000,0.654461,0.998305,1.000000
6,0.830267,0.666774,0.973557,0.866804
7,0.169733,0.333226,0.026443,0.133196
8,1.000000,0.508442,0.996970,1.000000
9,0.083367,0.213055,0.014177,0.066755


In [ ]:
pd.DataFrame(np.round(RES.T, 4))

,0,1,2,3
0,0.9405,0.6588,0.9896,0.9532
1,0.6535,0.4278,0.6524,0.6535
2,0.2871,0.2310,0.3372,0.2998
3,0.0595,0.1155,0.0093,0.0468
4,0.0000,0.2257,0.0011,0.0000
5,1.0000,0.6545,0.9983,1.0000
6,0.8303,0.6668,0.9736,0.8668
7,0.1697,0.3332,0.0264,0.1332
8,1.0000,0.5084,0.9970,1.0000
9,0.0834,0.2131,0.0142,0.0668


### Кросс-валидация и подбор оптимальных значений параметров

Как мы видели ранее, лучшие результаты по метрикам качества показывает алгоритм №3, при этом он намного быстрее алгоритмов №1 и №4. Поэтому в данном разделе мы будем искать оптимальное значение параметра k только для алгоритма "Мощность пересечения k ближайших объектов".

Предположим, что ошибки первого и второго рода приводят к одинаковым потерям, поэтому ориентируемся только на метрику Accuracy.

Для начала напишем функцию для кросс-валидации. Она будет принимать на вход массив данных и значение k для реализации алгоритма "Мощность пересечения k ближайших объектов".

In [ ]:
def cross_valid(data, k):
    accur = np.zeros(10)
    np.random.shuffle(data) # Перемешиваем объекты
    data_list = np.array_split(data, 10) # Делим выборку на 10 равных частей
    for i in range(10):
        i_data_list = tuple([data_list[j] for j in range(10) if j != i])
        A = np.concatenate(i_data_list, axis=0) # Объединяем все массивы, кроме i-го.
        X_train = A[:,0:9]
        y_train = A[:,9]
        X_test = data_list[i][:,0:9]
        y_test = data_list[i][:,9]
        y_pred = top_k_cardinality(X_train, y_train, X_test, k=k)
        accur[i] = get_metrics(y_test, y_pred)[0]
    return accur

Объединим все датасеты из тестовой выбоки в один и посмотрим на среднее значение точности прогнозов на кросс-валидации (количество строк равно 865).

In [ ]:
np.random.seed(42)
data = np.concatenate(tuple(list_test), axis=0)
for k in range(1, 20):
    print(k, np.round(np.mean(cross_valid(data, k)), 3))

1 0.488
2 0.513
3 0.659
4 0.79
5 0.87
6 0.933
7 0.961
8 0.968
9 0.98
10 0.984
11 0.991
12 0.989
13 0.991
14 0.987
15 0.987
16 0.986
17 0.989
18 0.981
19 0.98


Лучшие результаты показаны при $k=11$ и $k=13$. Видно, что при $k \geq 14$ среднее значение точности не превосходит 0.99 и убывает, поэтому разумно предположить, что мы нашли глобальный максимум точности.

### Свойства бинарных отношений

Проверим нашу модель на других данных. Вдруг модель хорошо работает только для крестиков-ноликов?

Во время выполнения первого домашнего задания по этому курсу я написал функции, проверяющие, является ли бинарное отношение асимметричным и является ли отношение транзитивным.

Давайте с их помощью сгенерируем 500 асимметричных и 500 не асимметричных бинарных отношений, которые будет задавать матрицей 3x3. 

In [ ]:
def is_asymmetric(M):
    if np.trace(M.dot(M)) > 0:
        return False
    return True

In [ ]:
np.random.seed(42)
data = []
num_elements = 0
while num_elements < 500:
    rand_array = stats.bernoulli(0.5).rvs(9)
    A = np.array(np.array_split(rand_array, 3))
    if is_asymmetric(A):
        data.append(rand_array)
        num_elements += 1
while num_elements < 1000:
    rand_array = stats.bernoulli(0.5).rvs(9)
    A = np.array(np.array_split(rand_array, 3))
    if not is_asymmetric(A):
        data.append(rand_array)
        num_elements += 1
data = np.array(data)
data = data.T
data = [list(arr) for arr in list(data)]
class_ = []
for i in range(500):
    class_.append('positive')
for i in range(500):
    class_.append('negative')
data.append(class_)
data = np.array(data)
data = data.T
data

array([['0', '0', '1', ..., '1', '0', 'positive'],
       ['0', '0', '1', ..., '1', '0', 'positive'],
       ['0', '0', '1', ..., '0', '0', 'positive'],
       ...,
       ['1', '0', '1', ..., '1', '1', 'negative'],
       ['0', '0', '1', ..., '1', '1', 'negative'],
       ['1', '1', '1', ..., '1', '1', 'negative']], dtype='<U11')

In [ ]:
len(data), len(data[0])

(1000, 10)

In [ ]:
np.random.seed(42)
for k in range(1, 20):
    print(k, np.round(np.mean(cross_valid(data, k)), 3))

1 1.0
2 1.0
3 0.999
4 0.998
5 0.996
6 0.98
7 0.976
8 0.956
9 0.95
10 0.932
11 0.925
12 0.909
13 0.9
14 0.889
15 0.886
16 0.883
17 0.885
18 0.883
19 0.884


А теперь проверим, как алгоритм справляется с проверкой на транзитивность.

In [ ]:
def is_transitive(M, n = 3):
    for i in range(n):
        for j in range(n):
            if M[i][j] == 1:
                for k in range(n):
                    if M[j][k] == 1:
                        if M[i][k] == 0:
                            return False
    return True


In [ ]:
np.random.seed(42)
data = []
num_elements = 0
while num_elements < 500:
    rand_array = stats.bernoulli(0.5).rvs(9)
    A = np.array(np.array_split(rand_array, 3))
    if is_transitive(A):
        data.append(rand_array)
        num_elements += 1
while num_elements < 1000:
    rand_array = stats.bernoulli(0.5).rvs(9)
    A = np.array(np.array_split(rand_array, 3))
    if not is_transitive(A):
        data.append(rand_array)
        num_elements += 1
data = np.array(data)
data = data.T
data = [list(arr) for arr in list(data)]
class_ = []
for i in range(500):
    class_.append('positive')
for i in range(500):
    class_.append('negative')
data.append(class_)
data = np.array(data)
data = data.T
data

array([['1', '1', '0', ..., '0', '1', 'positive'],
       ['0', '0', '1', ..., '0', '1', 'positive'],
       ['0', '1', '1', ..., '0', '0', 'positive'],
       ...,
       ['0', '1', '1', ..., '0', '0', 'negative'],
       ['1', '0', '1', ..., '1', '0', 'negative'],
       ['1', '1', '0', ..., '1', '0', 'negative']], dtype='<U11')

In [ ]:
np.random.seed(42)
for k in range(1, 20):
    print(k, np.round(np.mean(cross_valid(data, k)), 3))

1 0.964
2 0.965
3 0.973
4 0.97
5 0.979
6 0.965
7 0.976
8 0.968
9 0.966
10 0.954
11 0.946
12 0.947
13 0.946
14 0.942
15 0.95
16 0.943
17 0.945
18 0.946
19 0.949


### Выводы

Как можно заметить, для разных задач оптимальными являются разные значения параметра k.

Для определения итога партии в "крестики-нолики" оптимальными значениями были k=11 и k=13.

Для определения, является ли бинарное отношение асимметричным, оптимальными были k=1, k=2: при данных значениях параметров в наших экспериментах прогноз всегда был верным.

Для определения, является ли бинарное отношение транзитивным, оптимальным оказалось k=5.

Получается, что для каждой задачи оптимальное значение k может быть разным. В любом случае, метод показывает очень достойные результаты, в большинстве экспериментов точность прогноза превышает 95%.